In [3]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [88]:
med = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE >= '2021-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEDICAL_CLAIM_ID, 
       M.MEMBER_ID, 
       M.PAID_DATE, 
	   YEAR(M.PAID_DATE) AS PAID_YEAR,
       M.SERVICE_DATE,
YEAR(M.SERVICE_DATE) AS SERVICE_YEAR,
       M.MEDICAL_PAID_AMOUNT, 
       M.IS_TELEMEDICINE, 
       M.IS_ER_AVOIDABLE, 
M.DIAGNOSIS_DESC_ICD10_1 AS PRIMARY_DX,
       M.PROCEDURE_DESC, 
       M.ARTTOS_V2_L1, 
       M.ARTTOS_V2_L3, 
M.DX_IS_CHRONIC,
       M.ICD10_CHAPTER, 
       M.ICD10_CATEGORY, 
       M.ICD10_SECTION, 
       M.MEG_EPISODE_DESCRIPTION, 
       M.EMPLOYER_NAME, 
       M.OP_SURG_INC,
       M.IS_PCP_VISIT, 
       M.INCLUDED_SPECIALIST, 
       M.ER_VISIT_FLAG, 
       M.IS_URGENT_CARE_VISIT, 
       M.IS_PREVENTIVE_VISIT, 
       M.IP_ADMIT_INC
FROM [Actuarial_AH].[dbo].[SN_Medical] M
JOIN 
Actuarial_AH.dbo.SN_Member MB ON MB.MEMBER_ID = M.MEMBER_ID
WHERE M.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND MB.MONTH_KEY = '2024-12-01' 
AND MB.MEDICAL_ENROLLMENT_STATUS = 3
AND MB.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MB.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND M.ICD10_SECTION LIKE '%Overweight%'
"""
med = pd.read_sql(med, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_31864\3391108355.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  med = pd.read_sql(med, conn)


In [89]:
rx = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE BETWEEN '2021-01-01' AND '2024-12-31'
)
SELECT R.RX_CLAIM_ID, 
       R.MEMBER_ID, 
       R.PAID_DATE, 
       R.SERVICE_DATE, 
       R.DRUG_NAME_GENERIC,
	   R.DRUG_NAME_PREFERRED,
	   R.RX_PAID_AMOUNT,
	   R.RX_SCRIPT_COUNT,
	   R.ART_DRUGGRPS_L1,
	   R.ART_DRUGGRPS_L2,
	   YEAR(R.PAID_DATE) AS PAID_YEAR,
YEAR(R.SERVICE_DATE) AS SERVICE_YEAR
FROM [Actuarial_AH].[dbo].[SN_Rx] R
JOIN 
Actuarial_AH.dbo.SN_Member M ON R.MEMBER_ID = M.MEMBER_ID
JOIN Actuarial_AH.dbo.SN_Medical MD ON MD.MEMBER_ID = M.MEMBER_ID
WHERE R.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND M.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND MD.ICD10_SECTION LIKE '%Overweight%'
"""
rx = pd.read_sql(rx, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_31864\628379123.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rx = pd.read_sql(rx, conn)


In [90]:
member = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEMBER_ID,
       M.MEMBER_STATUS,
       M.EMPLOYER_NAME,
       M.HOME_STATE,
	   CASE 
    WHEN M.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
    WHEN M.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
    WHEN M.MEMBER_RELATIONSHIP = 2 THEN 'Dependent'
    WHEN M.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
    WHEN M.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS MEMBER_RELATIONSHIP,
CASE 
        WHEN M.GENDER = 0 THEN 'Female'
        WHEN M.GENDER = 1 THEN 'Male'
        WHEN M.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    DATEDIFF(YEAR, CONVERT(DATE, M.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age,
	CASE 
    WHEN M.BIRTH_YEAR >= 1946 AND M.BIRTH_YEAR < 1965 THEN 'Baby Boomers'
	WHEN M.BIRTH_YEAR >= 1965 AND M.BIRTH_YEAR < 1981 THEN 'Generation X'
	WHEN M.BIRTH_YEAR >= 1981 AND M.BIRTH_YEAR < 1997 THEN 'Millenials'
	WHEN M.BIRTH_YEAR >= 1997 AND M.BIRTH_YEAR < 2012 THEN 'Generation Z'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS GENERATIONS
FROM [Actuarial_AH].[dbo].[SN_MEMBER] M
JOIN Actuarial_AH.dbo.SN_Medical MD ON M.MEMBER_ID = MD.MEMBER_ID
WHERE M.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MD.ICD10_SECTION LIKE '%Overweight%'


"""
member = pd.read_sql(member, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_31864\1707225796.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  member = pd.read_sql(member, conn)


In [91]:
member

,MEMBER_ID,MEMBER_STATUS,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,mma-ad8147cdc4bc477fa0ee15a9ef25134d,0.00,EVANGELICAL GOOD SAMARITAN,MN,Subscriber,Female,61,Baby Boomers
1,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,0.00,EA SWEEN COMPANY,MN,Subscriber,Female,43,Millenials
2,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,0.00,EA SWEEN COMPANY,MN,Subscriber,Female,43,Millenials
3,mma-4019a5230a4f4b9c87bbbb1347cc44cc,0.00,SARGENTO FOODS,WI,Dependent,Male,21,Generation Z
4,mma-4019a5230a4f4b9c87bbbb1347cc44cc,0.00,SARGENTO FOODS,WI,Dependent,Male,21,Generation Z
...,...,...,...,...,...,...,...,...
154559,mma-8ea9ccb296b947a795ce5d17ea6ef2c7,2.00,MIAMI BEACH FRATERNAL ORDER OF POLICE,FL,Subscriber,Female,69,Baby Boomers
154560,mma-8ea9ccb296b947a795ce5d17ea6ef2c7,2.00,MIAMI BEACH FRATERNAL ORDER OF POLICE,FL,Subscriber,Female,69,Baby Boomers
154561,mma-8ea9ccb296b947a795ce5d17ea6ef2c7,2.00,MIAMI BEACH FRATERNAL ORDER OF POLICE,FL,Subscriber,Female,69,Baby Boomers
154562,mma-3a2003af67dc4d2c99b005afe647a135,0.00,GRIFOLS,NC,Dependent,Male,13,Unknown


In [92]:
med

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,EMPLOYER_NAME,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,16189405.00,mma-0008856832474235be7f58214e0cabc3,2022-10-21,2022,2022-10-10,2022,0.00,0.00,1.00,Other obesity,...,Overweight and obesity,Overweight and Obesity,TRAVEL CENTERS OF AMERICA,0,0,0,0,0,0,0
1,16189405.00,mma-0008856832474235be7f58214e0cabc3,2022-10-21,2022,2022-10-10,2022,0.00,0.00,1.00,Other obesity,...,Overweight and obesity,Overweight and Obesity,TRAVEL CENTERS OF AMERICA,0,0,0,0,0,0,0
2,16189405.00,mma-0008856832474235be7f58214e0cabc3,2022-10-21,2022,2022-10-10,2022,0.00,0.00,1.00,Other obesity,...,Overweight and obesity,Overweight and Obesity,TRAVEL CENTERS OF AMERICA,0,0,0,0,0,0,0
3,16189405.00,mma-0008856832474235be7f58214e0cabc3,2022-10-21,2022,2022-10-10,2022,0.00,0.00,1.00,Other obesity,...,Overweight and obesity,Overweight and Obesity,TRAVEL CENTERS OF AMERICA,0,0,0,0,0,0,0
4,5406378.00,mma-0016d31974914c4fb3c1c3448659448b,2024-10-11,2024,2024-09-27,2024,11.53,0.00,1.00,Overweight,...,Overweight and obesity,Overweight and Obesity,GRIFOLS,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105223,10549716.00,mma-ff52eaaf4b0740cbb781e7489be193af,2023-12-29,2023,2023-04-09,2023,2.66,0.00,1.00,"Obesity, unspecified",...,Overweight and obesity,Overweight and Obesity,EVANGELICAL GOOD SAMARITAN,0,0,0,0,0,0,0
105224,5310109.00,mma-ff858bc3d1c84fe2af64266686fee7af,2021-02-28,2021,2021-02-01,2021,65.56,1.00,1.00,Other obesity,...,Overweight and obesity,Overweight and Obesity,COBORNS,0,0,0,0,0,0,0
105225,10249658.00,mma-ff858bc3d1c84fe2af64266686fee7af,2021-04-30,2021,2021-04-07,2021,56.90,1.00,1.00,Other obesity,...,Overweight and obesity,Overweight and Obesity,COBORNS,0,0,0,0,0,0,0
105226,4213798.00,mma-ffcc54f657b746e9a0617f95e907fd9c,2024-01-04,2024,2023-12-26,2023,106.00,0.00,1.00,Morbid (severe) obesity due to excess calories,...,Overweight and obesity,Overweight and Obesity,EMBRAER AIRCRAFT HOLDINGS,0,0,0,0,0,0,0


In [93]:
rx_wla = rx[(rx['ART_DRUGGRPS_L2'] == 'Weight Loss Agents') | (rx['ART_DRUGGRPS_L1'] == 'GLP-1 Agonists')]
rx_wla.DRUG_NAME_PREFERRED.unique()

array(['Saxenda'], dtype=object)

In [94]:
member = member.drop_duplicates()
member.reset_index(inplace=True)


In [95]:
med.MEMBER_ID.nunique()

15472

In [96]:
rx.MEMBER_ID.nunique()

19556

In [133]:
med_agg = med.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='MEDICAL_PAID_AMOUNT', aggfunc='sum')
med_agg.reset_index(inplace=True)
med_agg.columns.name = None
med_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in med_agg.columns]
med_agg.rename(columns={'2021': 'Med_2021', '2022': 'Med_2022', '2023': 'Med_2023', '2024': 'Med_2024'}, inplace=True)

In [134]:
med_agg1 = med_agg
med_agg1

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,129.38,NaN,NaN,NaN
1,mma-00024d2d45224e76915920b058364ff1,NaN,142.76,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,278.02,NaN,NaN,NaN
3,mma-0008856832474235be7f58214e0cabc3,NaN,0.00,NaN,NaN
4,mma-000b1ae322e642bab96c125e11666661,68.49,67.19,NaN,75.86
...,...,...,...,...,...
15467,mma-ffde1dff43d644b6b68ce30172442972,NaN,NaN,NaN,1087.43
15468,mma-ffe711349c65480c9740ba217d4ad2cc,20.53,NaN,NaN,NaN
15469,mma-ffe7972f15694551af377f4666f57f60,NaN,NaN,0.00,NaN
15470,mma-fffb3143f0b84fa690e26308813fceb0,NaN,NaN,298.45,NaN


In [135]:
# Define inflation rates (as factors)
inflation_factors = {
    2021: 1.175, 
    2022: 1.117,   
    2023: 1.069, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in inflation_factors.items():
    column_name = f'Med_{year}'
    med_agg[column_name] = med_agg[column_name] * factor
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,152.02,NaN,NaN,NaN
1,mma-00024d2d45224e76915920b058364ff1,NaN,159.46,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,326.67,NaN,NaN,NaN
3,mma-0008856832474235be7f58214e0cabc3,NaN,0.00,NaN,NaN
4,mma-000b1ae322e642bab96c125e11666661,80.48,75.05,NaN,75.86
...,...,...,...,...,...
15467,mma-ffde1dff43d644b6b68ce30172442972,NaN,NaN,NaN,1087.43
15468,mma-ffe711349c65480c9740ba217d4ad2cc,24.12,NaN,NaN,NaN
15469,mma-ffe7972f15694551af377f4666f57f60,NaN,NaN,0.00,NaN
15470,mma-fffb3143f0b84fa690e26308813fceb0,NaN,NaN,319.04,NaN


In [136]:
med_agg.fillna(0, inplace=True)
med_agg.isnull().sum()

MEMBER_ID    0
Med_2021     0
Med_2022     0
Med_2023     0
Med_2024     0
dtype: int64

In [163]:
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,152.02,0.00,0.00,0.00,-152.02,-100.00,-45.61
1,mma-00024d2d45224e76915920b058364ff1,0.00,159.46,0.00,0.00,0.00,0.00,-15.95
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,326.67,0.00,0.00,0.00,-326.67,-100.00,-98.00
3,mma-0008856832474235be7f58214e0cabc3,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,mma-000b1ae322e642bab96c125e11666661,80.48,75.05,0.00,75.86,-4.62,-5.74,-8.89
...,...,...,...,...,...,...,...,...
15467,mma-ffde1dff43d644b6b68ce30172442972,0.00,0.00,0.00,1087.43,1087.43,inf,326.23
15468,mma-ffe711349c65480c9740ba217d4ad2cc,24.12,0.00,0.00,0.00,-24.12,-100.00,-7.24
15469,mma-ffe7972f15694551af377f4666f57f60,0.00,0.00,0.00,0.00,0.00,0.00,0.00
15470,mma-fffb3143f0b84fa690e26308813fceb0,0.00,0.00,319.04,0.00,0.00,0.00,31.90


In [164]:
med_agg['Med_Delta'] = med_agg['Med_2024'] - med_agg['Med_2021']
med_agg['Med_%_Change'] =(( med_agg['Med_2024'] - med_agg['Med_2021'])/ med_agg['Med_2021']) * 100
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,152.02,0.00,0.00,0.00,-152.02,-100.00,-45.61
1,mma-00024d2d45224e76915920b058364ff1,0.00,159.46,0.00,0.00,0.00,NaN,-15.95
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,326.67,0.00,0.00,0.00,-326.67,-100.00,-98.00
3,mma-0008856832474235be7f58214e0cabc3,0.00,0.00,0.00,0.00,0.00,NaN,0.00
4,mma-000b1ae322e642bab96c125e11666661,80.48,75.05,0.00,75.86,-4.62,-5.74,-8.89
...,...,...,...,...,...,...,...,...
15467,mma-ffde1dff43d644b6b68ce30172442972,0.00,0.00,0.00,1087.43,1087.43,inf,326.23
15468,mma-ffe711349c65480c9740ba217d4ad2cc,24.12,0.00,0.00,0.00,-24.12,-100.00,-7.24
15469,mma-ffe7972f15694551af377f4666f57f60,0.00,0.00,0.00,0.00,0.00,NaN,0.00
15470,mma-fffb3143f0b84fa690e26308813fceb0,0.00,0.00,319.04,0.00,0.00,NaN,31.90


In [165]:
med_agg.fillna(0, inplace=True)
med_agg.isnull().sum()

MEMBER_ID       0
Med_2021        0
Med_2022        0
Med_2023        0
Med_2024        0
Med_Delta       0
Med_%_Change    0
Med_Slope       0
dtype: int64

In [166]:
import numpy as np

# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in med_agg.iterrows():
    costs = np.array([row['Med_2021'], row['Med_2022'], row['Med_2023'], row['Med_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
med_agg['Med_Slope'] = slope_list
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,152.02,0.00,0.00,0.00,-152.02,-100.00,-45.61
1,mma-00024d2d45224e76915920b058364ff1,0.00,159.46,0.00,0.00,0.00,0.00,-15.95
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,326.67,0.00,0.00,0.00,-326.67,-100.00,-98.00
3,mma-0008856832474235be7f58214e0cabc3,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,mma-000b1ae322e642bab96c125e11666661,80.48,75.05,0.00,75.86,-4.62,-5.74,-8.89
...,...,...,...,...,...,...,...,...
15467,mma-ffde1dff43d644b6b68ce30172442972,0.00,0.00,0.00,1087.43,1087.43,inf,326.23
15468,mma-ffe711349c65480c9740ba217d4ad2cc,24.12,0.00,0.00,0.00,-24.12,-100.00,-7.24
15469,mma-ffe7972f15694551af377f4666f57f60,0.00,0.00,0.00,0.00,0.00,0.00,0.00
15470,mma-fffb3143f0b84fa690e26308813fceb0,0.00,0.00,319.04,0.00,0.00,0.00,31.90


In [167]:
rx_agg = rx.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='RX_PAID_AMOUNT', aggfunc='sum')
rx_agg.reset_index(inplace=True)
rx_agg.columns.name = None
rx_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in rx_agg.columns]
rx_agg.rename(columns={'2021': 'Rx_2021', '2022': 'Rx_2022', '2023': 'Rx_2023', '2024': 'Rx_2024'}, inplace=True)
rx_agg1 = rx_agg
rx_agg1

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,1248.02,417.37,369.43,671.02
1,mma-00024d2d45224e76915920b058364ff1,42.40,0.00,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,1968.26,1783.30,10715.61
3,mma-0007e20948f4487d97f4eb2981613ea1,417.20,143.25,43.80,189.95
4,mma-0008856832474235be7f58214e0cabc3,62.00,0.00,0.00,82.36
...,...,...,...,...,...
19551,mma-ffe7972f15694551af377f4666f57f60,0.00,2194.32,1220.24,0.00
19552,mma-ffe9040f10b9429eb7ce09ceaaf1e158,677.35,943.25,762.00,546.50
19553,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,97.96,531.72,578.40
19554,mma-fffb3143f0b84fa690e26308813fceb0,104.12,693.53,2549.36,877.94


In [168]:
# Define inflation rates (as factors)
rx_inflation_factors = {
    2021: 1.31, 
    2022: 1.20,   
    2023: 1.103, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in rx_inflation_factors.items():
    column_name = f'Rx_{year}'
    rx_agg[column_name] = rx_agg[column_name] * factor
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02
1,mma-00024d2d45224e76915920b058364ff1,55.54,0.00,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,2361.91,1966.98,10715.61
3,mma-0007e20948f4487d97f4eb2981613ea1,546.53,171.90,48.31,189.95
4,mma-0008856832474235be7f58214e0cabc3,81.22,0.00,0.00,82.36
...,...,...,...,...,...
19551,mma-ffe7972f15694551af377f4666f57f60,0.00,2633.18,1345.92,0.00
19552,mma-ffe9040f10b9429eb7ce09ceaaf1e158,887.33,1131.90,840.49,546.50
19553,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,117.55,586.49,578.40
19554,mma-fffb3143f0b84fa690e26308813fceb0,136.40,832.24,2811.94,877.94


In [169]:
rx_agg.fillna(0, inplace=True)
rx_agg.isnull().sum()

MEMBER_ID    0
Rx_2021      0
Rx_2022      0
Rx_2023      0
Rx_2024      0
dtype: int64

In [170]:
rx_agg['Rx_Delta'] = rx_agg['Rx_2024'] - rx_agg['Rx_2021']
rx_agg['Rx_%_Change'] =((rx_agg['Rx_2024'] - rx_agg['Rx_2021'])/ rx_agg['Rx_2021']) * 100
# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in rx_agg.iterrows():
    costs = np.array([row['Rx_2021'], row['Rx_2022'], row['Rx_2023'], row['Rx_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
rx_agg['Rx_Slope'] = slope_list
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02,-963.89,-58.96,-298.50
1,mma-00024d2d45224e76915920b058364ff1,55.54,0.00,0.00,0.00,-55.54,-100.00,-16.66
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,2361.91,1966.98,10715.61,10715.61,inf,3175.19
3,mma-0007e20948f4487d97f4eb2981613ea1,546.53,171.90,48.31,189.95,-356.58,-65.24,-119.33
4,mma-0008856832474235be7f58214e0cabc3,81.22,0.00,0.00,82.36,1.14,1.40,0.34
...,...,...,...,...,...,...,...,...
19551,mma-ffe7972f15694551af377f4666f57f60,0.00,2633.18,1345.92,0.00,0.00,NaN,-128.73
19552,mma-ffe9040f10b9429eb7ce09ceaaf1e158,887.33,1131.90,840.49,546.50,-340.83,-38.41,-131.39
19553,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,117.55,586.49,578.40,578.40,inf,220.41
19554,mma-fffb3143f0b84fa690e26308813fceb0,136.40,832.24,2811.94,877.94,741.54,543.66,420.43


In [171]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [172]:
rx_agg.replace(np.inf, 0, inplace=True)
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02,-963.89,-58.96,-298.50
1,mma-00024d2d45224e76915920b058364ff1,55.54,0.00,0.00,0.00,-55.54,-100.00,-16.66
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,2361.91,1966.98,10715.61,10715.61,0.00,3175.19
3,mma-0007e20948f4487d97f4eb2981613ea1,546.53,171.90,48.31,189.95,-356.58,-65.24,-119.33
4,mma-0008856832474235be7f58214e0cabc3,81.22,0.00,0.00,82.36,1.14,1.40,0.34
...,...,...,...,...,...,...,...,...
19551,mma-ffe7972f15694551af377f4666f57f60,0.00,2633.18,1345.92,0.00,0.00,NaN,-128.73
19552,mma-ffe9040f10b9429eb7ce09ceaaf1e158,887.33,1131.90,840.49,546.50,-340.83,-38.41,-131.39
19553,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,117.55,586.49,578.40,578.40,0.00,220.41
19554,mma-fffb3143f0b84fa690e26308813fceb0,136.40,832.24,2811.94,877.94,741.54,543.66,420.43


In [173]:
member_mrg = member.merge(med_agg, on='MEMBER_ID')
member_mrg = member_mrg.merge(rx_agg, on='MEMBER_ID')                                     
member_mrg       

,index,MEMBER_ID,MEMBER_STATUS,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,0,mma-ad8147cdc4bc477fa0ee15a9ef25134d,0.00,EVANGELICAL GOOD SAMARITAN,MN,Subscriber,Female,61,Baby Boomers,209.81,...,-209.81,-100.00,-62.94,109.40,0.00,0.00,6.78,-102.62,-93.80,-30.79
1,1,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,0.00,EA SWEEN COMPANY,MN,Subscriber,Female,43,Millenials,0.00,...,0.00,0.00,-0.99,0.00,108.19,0.00,0.00,0.00,NaN,-10.82
2,3,mma-4019a5230a4f4b9c87bbbb1347cc44cc,0.00,SARGENTO FOODS,WI,Dependent,Male,21,Generation Z,0.00,...,0.00,0.00,0.00,0.00,612.84,505.73,3.82,3.82,0.00,-9.57
3,5,mma-007ed462b23b4c82a8153a7cc91d2367,0.00,COBORNS,MN,Subscriber,Female,41,Millenials,0.00,...,0.00,0.00,0.60,82.10,0.00,0.00,0.00,-82.10,-100.00,-24.63
4,6,mma-221b37a7beb24bf18b15772668742d7f,0.00,GRIFOLS,NC,Subscriber,Female,57,Generation X,43.63,...,-43.63,-100.00,-13.09,0.00,0.00,0.00,1397.52,1397.52,0.00,419.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15154,154541,mma-6c893f56dc514080b33d36417b0435d3,0.00,ONE WEBASTO,MI,Spouse,Female,51,Generation X,0.00,...,0.00,0.00,-10.86,3292.66,773.38,624.70,1559.90,-1732.76,-52.62,-534.70
15155,154543,mma-ab3922da2eed4e469a0f47c57d752929,0.00,ALLTECH,MN,Spouse,Male,47,Generation X,372.66,...,-372.66,-100.00,-111.80,279.50,1030.97,470.14,505.20,225.70,80.75,11.63
15156,154546,mma-ce378a7b44c447e496496ec37337154e,0.00,ICU,CA,Subscriber,Male,49,Generation X,0.00,...,5.42,inf,1.63,0.00,336.00,0.00,613.06,613.06,0.00,150.32
15157,154562,mma-3a2003af67dc4d2c99b005afe647a135,0.00,GRIFOLS,NC,Dependent,Male,13,Unknown,0.00,...,132.53,inf,39.76,0.00,22.63,0.00,29.34,29.34,0.00,6.54


In [174]:
member1 = member_mrg.drop(columns=['MEMBER_STATUS', 'EMPLOYER_NAME'])
member1

,index,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,0,mma-ad8147cdc4bc477fa0ee15a9ef25134d,MN,Subscriber,Female,61,Baby Boomers,209.81,0.00,0.00,...,-209.81,-100.00,-62.94,109.40,0.00,0.00,6.78,-102.62,-93.80,-30.79
1,1,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,MN,Subscriber,Female,43,Millenials,0.00,9.86,0.00,...,0.00,0.00,-0.99,0.00,108.19,0.00,0.00,0.00,NaN,-10.82
2,3,mma-4019a5230a4f4b9c87bbbb1347cc44cc,WI,Dependent,Male,21,Generation Z,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,612.84,505.73,3.82,3.82,0.00,-9.57
3,5,mma-007ed462b23b4c82a8153a7cc91d2367,MN,Subscriber,Female,41,Millenials,0.00,0.00,5.95,...,0.00,0.00,0.60,82.10,0.00,0.00,0.00,-82.10,-100.00,-24.63
4,6,mma-221b37a7beb24bf18b15772668742d7f,NC,Subscriber,Female,57,Generation X,43.63,0.00,0.00,...,-43.63,-100.00,-13.09,0.00,0.00,0.00,1397.52,1397.52,0.00,419.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15154,154541,mma-6c893f56dc514080b33d36417b0435d3,MI,Spouse,Female,51,Generation X,0.00,108.57,0.00,...,0.00,0.00,-10.86,3292.66,773.38,624.70,1559.90,-1732.76,-52.62,-534.70
15155,154543,mma-ab3922da2eed4e469a0f47c57d752929,MN,Spouse,Male,47,Generation X,372.66,0.00,0.00,...,-372.66,-100.00,-111.80,279.50,1030.97,470.14,505.20,225.70,80.75,11.63
15156,154546,mma-ce378a7b44c447e496496ec37337154e,CA,Subscriber,Male,49,Generation X,0.00,0.00,0.00,...,5.42,inf,1.63,0.00,336.00,0.00,613.06,613.06,0.00,150.32
15157,154562,mma-3a2003af67dc4d2c99b005afe647a135,NC,Dependent,Male,13,Unknown,0.00,0.00,0.00,...,132.53,inf,39.76,0.00,22.63,0.00,29.34,29.34,0.00,6.54


In [175]:
med_util = med.groupby('MEMBER_ID').agg({
    'DX_IS_CHRONIC': 'max',
    'ER_VISIT_FLAG': 'sum',
    'IS_PCP_VISIT': 'sum',
    'IS_URGENT_CARE_VISIT': 'sum',
    'IS_PREVENTIVE_VISIT': 'sum',
    'IP_ADMIT_INC': 'sum'
}).reset_index()
med_util

,MEMBER_ID,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-0001ed26305b48f99696f11dfcea17b7,1.00,0,1,0,0,0
1,mma-00024d2d45224e76915920b058364ff1,1.00,0,0,0,1,0
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,1.00,0,0,0,0,0
3,mma-0008856832474235be7f58214e0cabc3,1.00,0,0,0,0,0
4,mma-000b1ae322e642bab96c125e11666661,1.00,0,3,0,0,0
...,...,...,...,...,...,...,...
15467,mma-ffde1dff43d644b6b68ce30172442972,1.00,0,5,0,0,0
15468,mma-ffe711349c65480c9740ba217d4ad2cc,1.00,0,0,0,0,0
15469,mma-ffe7972f15694551af377f4666f57f60,1.00,0,0,0,0,0
15470,mma-fffb3143f0b84fa690e26308813fceb0,1.00,0,1,0,0,0


In [176]:
merge_member = member1.merge(med_util, on='MEMBER_ID')
merge_member

,index,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,0,mma-ad8147cdc4bc477fa0ee15a9ef25134d,MN,Subscriber,Female,61,Baby Boomers,209.81,0.00,0.00,...,6.78,-102.62,-93.80,-30.79,1.00,0,0,0,0,0
1,1,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,MN,Subscriber,Female,43,Millenials,0.00,9.86,0.00,...,0.00,0.00,NaN,-10.82,1.00,0,0,0,0,0
2,3,mma-4019a5230a4f4b9c87bbbb1347cc44cc,WI,Dependent,Male,21,Generation Z,0.00,0.00,0.00,...,3.82,3.82,0.00,-9.57,1.00,0,0,0,0,0
3,5,mma-007ed462b23b4c82a8153a7cc91d2367,MN,Subscriber,Female,41,Millenials,0.00,0.00,5.95,...,0.00,-82.10,-100.00,-24.63,1.00,0,1,0,0,0
4,6,mma-221b37a7beb24bf18b15772668742d7f,NC,Subscriber,Female,57,Generation X,43.63,0.00,0.00,...,1397.52,1397.52,0.00,419.26,1.00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15154,154541,mma-6c893f56dc514080b33d36417b0435d3,MI,Spouse,Female,51,Generation X,0.00,108.57,0.00,...,1559.90,-1732.76,-52.62,-534.70,1.00,0,1,0,0,0
15155,154543,mma-ab3922da2eed4e469a0f47c57d752929,MN,Spouse,Male,47,Generation X,372.66,0.00,0.00,...,505.20,225.70,80.75,11.63,1.00,0,1,0,0,0
15156,154546,mma-ce378a7b44c447e496496ec37337154e,CA,Subscriber,Male,49,Generation X,0.00,0.00,0.00,...,613.06,613.06,0.00,150.32,1.00,0,0,0,0,0
15157,154562,mma-3a2003af67dc4d2c99b005afe647a135,NC,Dependent,Male,13,Unknown,0.00,0.00,0.00,...,29.34,29.34,0.00,6.54,1.00,0,1,0,0,0


In [178]:
merge_member.drop(columns='index', inplace=True)

In [182]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)
merge_member

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-ad8147cdc4bc477fa0ee15a9ef25134d,MN,Subscriber,Female,61,Baby Boomers,209.81,0.00,0.00,0.00,-209.81,-100.00,-62.94,109.40,0.00,0.00,6.78,-102.62,-93.80,-30.79,1.00,0,0,0,0,0
1,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,MN,Subscriber,Female,43,Millenials,0.00,9.86,0.00,0.00,0.00,0.00,-0.99,0.00,108.19,0.00,0.00,0.00,NaN,-10.82,1.00,0,0,0,0,0
2,mma-4019a5230a4f4b9c87bbbb1347cc44cc,WI,Dependent,Male,21,Generation Z,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,612.84,505.73,3.82,3.82,0.00,-9.57,1.00,0,0,0,0,0
3,mma-007ed462b23b4c82a8153a7cc91d2367,MN,Subscriber,Female,41,Millenials,0.00,0.00,5.95,0.00,0.00,0.00,0.60,82.10,0.00,0.00,0.00,-82.10,-100.00,-24.63,1.00,0,1,0,0,0
4,mma-221b37a7beb24bf18b15772668742d7f,NC,Subscriber,Female,57,Generation X,43.63,0.00,0.00,0.00,-43.63,-100.00,-13.09,0.00,0.00,0.00,1397.52,1397.52,0.00,419.26,1.00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15154,mma-6c893f56dc514080b33d36417b0435d3,MI,Spouse,Female,51,Generation X,0.00,108.57,0.00,0.00,0.00,0.00,-10.86,3292.66,773.38,624.70,1559.90,-1732.76,-52.62,-534.70,1.00,0,1,0,0,0
15155,mma-ab3922da2eed4e469a0f47c57d752929,MN,Spouse,Male,47,Generation X,372.66,0.00,0.00,0.00,-372.66,-100.00,-111.80,279.50,1030.97,470.14,505.20,225.70,80.75,11.63,1.00,0,1,0,0,0
15156,mma-ce378a7b44c447e496496ec37337154e,CA,Subscriber,Male,49,Generation X,0.00,0.00,0.00,5.42,5.42,inf,1.63,0.00,336.00,0.00,613.06,613.06,0.00,150.32,1.00,0,0,0,0,0
15157,mma-3a2003af67dc4d2c99b005afe647a135,NC,Dependent,Male,13,Unknown,0.00,0.00,0.00,132.53,132.53,inf,39.76,0.00,22.63,0.00,29.34,29.34,0.00,6.54,1.00,0,1,0,0,0


In [183]:
episodes = med.MEG_EPISODE_DESCRIPTION.value_counts()
episodes.head(10)

MEG_EPISODE_DESCRIPTION
Overweight and Obesity                                98696
OTHER                                                  4310
Encounter for Preventive Health Services                719
Essential Hypertension, Chronic Maintenance             560
Diabetes Mellitus Type 2 & Unspec Type Maintenance      456
Hypothyroidism                                          247
Osteoarthritis, Except Spine                            187
Other Arthropathies, Bone and Joint Disorders            32
Factors Influencing Health Status                         8
Depression                                                7
Name: count, dtype: int64

In [184]:
episodes_cost = med.groupby('MEG_EPISODE_DESCRIPTION')['MEDICAL_PAID_AMOUNT'].sum().reset_index()
episodes_cost.sort_values(by='MEDICAL_PAID_AMOUNT', ascending=False)

,MEG_EPISODE_DESCRIPTION,MEDICAL_PAID_AMOUNT
10,Overweight and Obesity,20998325.16
6,OTHER,629678.38
7,"Osteoarthritis, Except Spine",89111.97
2,Encounter for Preventive Health Services,66172.56
1,Diabetes Mellitus Type 2 & Unspec Type Mainten...,14044.13
3,"Essential Hypertension, Chronic Maintenance",13854.06
5,Hypothyroidism,5842.71
4,Factors Influencing Health Status,3176.45
0,Depression,2962.15
8,"Other Arthropathies, Bone and Joint Disorders",612.42


In [185]:
episodes_to_keep = ['Encounter for Preventive Health Services', 'Overweight and Obesity', 'Essential Hypertension, Chronic Maintenance', 
                    'Osteoarthritis, Except Spine', 'Other Arthropathies, Bone and Joint Disorders', 'Depression', 'Diabetes Mellitus Type 2 & Unspec Type Maintenance',
                   'Other Spinal and Back Disorders, Low Back', 'Hypothyroidism', 'Factors Influencing Health Status']
med1 = med
med1['MEG_EPISODE_DESCRIPTION'] = med['MEG_EPISODE_DESCRIPTION'].apply(lambda x: x if x in episodes_to_keep else 'OTHER')

In [186]:
med_meg = pd.pivot_table(
    med1,
    values='MEDICAL_PAID_AMOUNT',
    index='MEMBER_ID',
    columns='MEG_EPISODE_DESCRIPTION',
    aggfunc='count',
    fill_value=0
)
med_meg.reset_index()
med_meg.columns.name = None
med_meg.drop(columns='OTHER', inplace=True)
med_meg

,Depression,Diabetes Mellitus Type 2 & Unspec Type Maintenance,Encounter for Preventive Health Services,"Essential Hypertension, Chronic Maintenance",Factors Influencing Health Status,Hypothyroidism,"Osteoarthritis, Except Spine","Other Arthropathies, Bone and Joint Disorders","Other Spinal and Back Disorders, Low Back",Overweight and Obesity
MEMBER_ID,,,,,,,,,,
mma-0001ed26305b48f99696f11dfcea17b7,0,0,0,0,0,0,0,0,0,1
mma-00024d2d45224e76915920b058364ff1,0,0,1,0,0,0,0,0,0,0
mma-0006f22cc2ed4e6da73eb73991bbbf57,0,0,0,0,0,0,0,0,0,1
mma-0008856832474235be7f58214e0cabc3,0,0,0,0,0,0,0,0,0,4
mma-000b1ae322e642bab96c125e11666661,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...
mma-ffde1dff43d644b6b68ce30172442972,0,0,0,0,0,0,0,0,0,5
mma-ffe711349c65480c9740ba217d4ad2cc,0,0,0,0,0,0,0,0,0,4
mma-ffe7972f15694551af377f4666f57f60,0,0,0,0,0,0,0,0,0,8


In [187]:
member_merge_cond = merge_member.merge(med_meg, on='MEMBER_ID')
member_merge_cond

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Depression,Diabetes Mellitus Type 2 & Unspec Type Maintenance,Encounter for Preventive Health Services,"Essential Hypertension, Chronic Maintenance",Factors Influencing Health Status,Hypothyroidism,"Osteoarthritis, Except Spine","Other Arthropathies, Bone and Joint Disorders","Other Spinal and Back Disorders, Low Back",Overweight and Obesity
0,mma-ad8147cdc4bc477fa0ee15a9ef25134d,MN,Subscriber,Female,61,Baby Boomers,209.81,0.00,0.00,0.00,-209.81,-100.00,-62.94,109.40,0.00,0.00,6.78,-102.62,-93.80,-30.79,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,MN,Subscriber,Female,43,Millenials,0.00,9.86,0.00,0.00,0.00,0.00,-0.99,0.00,108.19,0.00,0.00,0.00,NaN,-10.82,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,mma-4019a5230a4f4b9c87bbbb1347cc44cc,WI,Dependent,Male,21,Generation Z,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,612.84,505.73,3.82,3.82,0.00,-9.57,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,mma-007ed462b23b4c82a8153a7cc91d2367,MN,Subscriber,Female,41,Millenials,0.00,0.00,5.95,0.00,0.00,0.00,0.60,82.10,0.00,0.00,0.00,-82.10,-100.00,-24.63,1.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,mma-221b37a7beb24bf18b15772668742d7f,NC,Subscriber,Female,57,Generation X,43.63,0.00,0.00,0.00,-43.63,-100.00,-13.09,0.00,0.00,0.00,1397.52,1397.52,0.00,419.26,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15154,mma-6c893f56dc514080b33d36417b0435d3,MI,Spouse,Female,51,Generation X,0.00,108.57,0.00,0.00,0.00,0.00,-10.86,3292.66,773.38,624.70,1559.90,-1732.76,-52.62,-534.70,1.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
15155,mma-ab3922da2eed4e469a0f47c57d752929,MN,Spouse,Male,47,Generation X,372.66,0.00,0.00,0.00,-372.66,-100.00,-111.80,279.50,1030.97,470.14,505.20,225.70,80.75,11.63,1.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3
15156,mma-ce378a7b44c447e496496ec37337154e,CA,Subscriber,Male,49,Generation X,0.00,0.00,0.00,5.42,5.42,inf,1.63,0.00,336.00,0.00,613.06,613.06,0.00,150.32,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15157,mma-3a2003af67dc4d2c99b005afe647a135,NC,Dependent,Male,13,Unknown,0.00,0.00,0.00,132.53,132.53,inf,39.76,0.00,22.63,0.00,29.34,29.34,0.00,6.54,1.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [188]:
member_merge_cond.describe()

C:\Users\U1229773\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\U1229773\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Depression,Diabetes Mellitus Type 2 & Unspec Type Maintenance,Encounter for Preventive Health Services,"Essential Hypertension, Chronic Maintenance",Factors Influencing Health Status,Hypothyroidism,"Osteoarthritis, Except Spine","Other Arthropathies, Bone and Joint Disorders","Other Spinal and Back Disorders, Low Back",Overweight and Obesity
count,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,13654.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00,15159.00
mean,44.66,400.40,434.72,386.95,338.87,-61.53,NaN,-23.24,18426.25,21927.75,25453.72,25887.84,7461.59,-inf,2591.08,1.00,0.01,0.93,0.00,0.16,0.51,0.00,0.03,0.05,0.04,0.00,0.02,0.01,0.00,0.00,6.40
std,15.02,3597.07,3571.15,3305.17,2923.19,4631.61,NaN,1480.71,159552.60,206351.98,292730.83,249647.79,238255.78,NaN,86790.05,0.04,0.43,2.42,0.09,1.33,3.41,0.02,0.30,0.42,0.31,0.04,0.30,0.64,0.11,0.04,12.10
min,4.00,-1398.83,-2135.57,-30660.47,-7336.26,-91053.12,-inf,-27291.16,-36558.98,-9057.85,-5457.25,-21042.98,-8150200.02,-inf,-2569101.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,36.00,0.00,0.00,0.00,0.00,0.00,0.00,-6.43,13.13,26.29,6.34,3.21,-580.03,-86.07,-204.79,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
50%,46.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,391.41,424.82,402.28,370.09,0.00,-9.88,-0.31,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00
75%,56.00,0.00,0.00,14.23,38.61,29.34,NaN,20.81,2625.99,2895.96,3128.71,3264.46,615.50,64.17,232.68,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.00
max,87.00,91053.12,64410.13,82483.51,70828.57,70828.57,inf,21504.72,9142459.77,10965077.04,25228207.34,19336560.40,19214650.23,1080133364827837824.00,7440455.72,1.00,32.00,86.00,6.00,62.00,114.00,2.00,14.00,36.00,15.00,3.00,22.00,51.00,12.00,4.00,250.00


In [189]:
drugs = rx.groupby(['ART_DRUGGRPS_L1', 'ART_DRUGGRPS_L2', 'DRUG_NAME_PREFERRED'])['RX_PAID_AMOUNT'].sum().reset_index()
drugs.sort_values(by='RX_PAID_AMOUNT', ascending=False, inplace=True)
drugs

,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT
233,CNS Agents,Anti-Rheumatics/Immune Modulators,Humira (2 Pen),139049491.90
231,CNS Agents,Anti-Rheumatics/Immune Modulators,Enbrel SureClick,50532384.50
2711,Topical Agents,Psoriasis Agents,Taltz,45694352.85
2710,Topical Agents,Psoriasis Agents,Stelara,41496017.80
2707,Topical Agents,Psoriasis Agents,Skyrizi Pen,40127120.12
...,...,...,...,...
1690,Lipid Agents,Niacin Agents,Niacin (Antihyperlipidemic),-853.06
715,CNS Agents,Weight Loss Agents,Saxenda,-1234.35
2040,Nutrional,Prenatal Vitamins,PreGenna,-1483.64
1784,Medical Supplies,Medical Supplies,Contour Next EZ,-45210.55


In [191]:
member_merge_cond.to_csv('no-glp1-members.csv', index=False)